In [4]:
import pandas as pd
from scipy.io import loadmat
import numpy as np
import scipy.stats as s
import os
import matplotlib.pyplot as plt
import cv2

# preparing of training and testing datasets

In [5]:
trainingpath="datasets/DevanagariHandwrittenCharacterDataset//Train"
testingpath="datasets/DevanagariHandwrittenCharacterDataset//Test"
labels=os.listdir(trainingpath)
labels

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'adna',
 'ba',
 'bha',
 'cha',
 'chha',
 'chhya',
 'da',
 'daa',
 'dha',
 'dhaa',
 'ga',
 'gha',
 'gya',
 'ha',
 'ja',
 'jha',
 'ka',
 'kha',
 'kna',
 'la',
 'ma',
 'motosaw',
 'na',
 'pa',
 'patalosaw',
 'petchiryakha',
 'pha',
 'ra',
 'taamatar',
 'tabala',
 'tha',
 'thaa',
 'tra',
 'waw',
 'yaw',
 'yna']

In [6]:
len(labels)
i1 = os.listdir(trainingpath+"//"+labels[0])
len(i1)

img = cv2.imread(trainingpath+"//"+labels[0]+"//"+i1[0],0)
img.shape

(32, 32)

In [7]:

# imagesList = os.listdir(trainingpath+"//"+labels[1])
# img = cv2.imread(trainingpath+"//"+labels[1]+"//"+imagesList[1],0)

In [ ]:
df_train=[]
label_list=[]
for eachLabel in labels:
    imagesList = os.listdir(trainingpath+"//"+eachLabel)
    for image in imagesList:
        img = cv2.imread(trainingpath+"//"+eachLabel+"//"+image,0)
        df_train.append(img)
        label_list.append(eachLabel)
    print(eachLabel+"  done")
df_train=np.array(df_train)
label_list=np.array(label_list)


0  done
1  done


In [130]:
df_train=df_train.reshape(df_train.shape[0],df_train.shape[1]*df_train.shape[2])

df_train=pd.DataFrame(df_train)
label_df = pd.DataFrame(data = label_list)
df_train=pd.concat([df_train,label_df],axis = 1,ignore_index= True)

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,character_10_yna,character_10_yna
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,character_10_yna,character_10_yna
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,character_10_yna,character_10_yna
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,character_10_yna,character_10_yna
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,character_10_yna,character_10_yna


In [73]:
df_test=[]
label_list=[]
for eachLabel in labels:
    imagesList = os.listdir(testingpath+"//"+eachLabel)
    for image in imagesList:
        img = cv2.imread(testingpath+"//"+eachLabel+"//"+image,0)
        df_test.append(img)
        label_list.append(eachLabel)
df_test=np.array(df_test)
label_list=np.array(label_list)
df_test=df_test.reshape(df_test.shape[0],df_test.shape[1]*df_test.shape[2])

df_test=pd.df_test(df_test)
label_df = pd.DataFrame(data = label_list)
df_test=pd.concat([df_test,label_df],axis = 1,ignore_index= True)

# Principal Component Analysis

In [86]:
def PCA(data_to_train,data_to_test):
    cov=np.cov(data_to_train.T)
    eigenValues,eigenVectors=np.linalg.eig(cov)
    eigen_pairs={}
    for i in range(len(eigenValues)):
        eigen_pairs[eigenValues[i]]=eigenVectors[i]
    total_variance=sum(eigenValues)
    percentageVariance=[(variance/total_variance) for variance in sorted(eigenValues,reverse=True) ]
    accumulativeVariance=[]
    accumulativeVariance=np.cumsum(percentageVariance)
    for i in range(len(accumulativeVariance)):
        if accumulativeVariance[i]>=.95:
            breaking_index=i
            break
    projection=[]
    for i in sorted(eigenValues,reverse=True):
        projection.append(np.real(eigen_pairs[i][:breaking_index]))
    projection=np.array(projection)
    X_Train_Data_PCA = data_to_train.dot(projection)

    X_Test_Data_PCA = data_to_test.dot(projection)
    
    return X_Train_Data_PCA,X_Test_Data_PCA

In [119]:

x_Train_Data = df_train.iloc[:,:-1]
y_Train_Data = df_train.iloc[:,-1]

x_Test_Data = df_test.iloc[:,:-1]
y_Test_Data = df_test.iloc[:,-1].values

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc1=StandardScaler()
x_Train_Data=sc.fit_transform(x_Train_Data)
x_Test_Data=sc.transform(x_Test_Data)


character_wise_data={}
for label in labels:
    character_wise_data[label]=df_train[df_train['label']==label].iloc[:,:-1]

In [120]:

pcaplist=[]
for label in labels:
    pcaplist.append(character_wise_data[label].shape[0]/df_train.shape[0])

In [121]:
sum(pcaplist)

0.9999999999999994

In [122]:
from math import log
posterior_prob=[]
for label,i in zip(labels,range(len(labels))):
    x_Train_Data_PCA,x_Test_Data_PCA=PCA(character_wise_data[label],x_Test_Data)
    Mean=np.mean(x_Train_Data_PCA,axis=0)
    Cov=np.cov(x_Train_Data_PCA.T)
     
    prob=log(pcaplist[i])+s.multivariate_normal.logpdf(x_Test_Data_PCA,Mean,Cov)
    
    posterior_prob.append(prob)

In [123]:
posterior_prob = (np.array(posterior_prob)).T
posterior_prob.shape

(13800, 46)

In [127]:
count=0
for example in range(len(x_Test_Data)):
    print(labels[np.argmax(posterior_prob[example])])
    print(y_Test_Data[example]+"\n")
    if labels[np.argmax(posterior_prob[example])]==y_Test_Data[example]:
        count+=1


digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
character_10_yna

digit_0
char


digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0
character_11_taamatar

digit_0

character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13_daa

digit_0
character_13

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

digit_0
character_15_adna

d

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
character_16_tabala

digit_0
charac

...


digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit_2

digit_0
digit

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_4

digit_0
digit_

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_6

digit_0
digit_

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_8

digit_0
digit_


digit_0
digit_9

digit_0
digit_9

digit_0
digit_9

digit_0
digit_9

digit_0
digit_9

digit_0
digit_9

digit_0
digit_9



In [126]:
accuracy=count*100/len(y_Test_Data)
print("Accuracy: ",round(accuracy),"%")

Accuracy:  2 %
